In [1]:
from utilities import *
from astropy.io import fits
from astropy.table import QTable
import glob
import radio_beam
import sys

source='DSi'
fnum=fields[source]

linemodelhome='/blue/adamginsburg/d.jeff/SgrB2DS_COMAnalysis/linemodels/'
linemodelversion='firstrelease'
linemodelpath=linemodelhome+f'{linemodelversion}/{source}/'

targetmolecule='C2H5OH'
targetmoleculepath=linemodelpath+f'{targetmolecule}.fits'
targetmoltable=QTable.read(targetmoleculepath)

othermoleculepaths=glob.glob(linemodelpath+'*.fits')
othermoleculepaths.remove(targetmoleculepath)

ch3ohpath=ch3oh_sourcedict[source]
targetpix=pixdict[source]
origsourcepath=f'/blue/adamginsburg/d.jeff/SgrB2DSreorg/field{fnum}/CH3OH/{source}{ch3ohpath}'
contpath=origsourcepath+'reprojectedcontinuum.fits'

fwhmpath=glob.glob(origsourcepath+'*fwhm*')[0]
fwhmmap=fits.getdata(fwhmpath)*u.km/u.s
fwhm_at_pix=fwhmmap[targetpix[0],targetpix[1]]

reprojcontfits=fits.open(contpath)
reprojcont=reprojcontfits[0].data*u.Jy
reprojcontrestfreq=225*u.GHz#manual addition 11/9/2022, wiggle room w/i GHz
cntmbeam=radio_beam.Beam.from_fits_header(reprojcontfits[0].header)
reprojcont_K=reprojcont.to('K',cntmbeam.jtok_equiv(reprojcontrestfreq))
continuumlevel=reprojcont_K[targetpix[0],targetpix[1]]

In [10]:
safelines=[]
minimumcontaminantthreshold=0.1#If contaminant lines have more than this times the target line's flux, the target line is considered contaminated and removed
firstline=True
for targetline in targetmoltable:
    flux=targetline['ModelBrightness']
    reffreq=targetline['ReferenceFrequency']
    qn=targetline['QNs']
    if flux <= continuumlevel:
        continue
    else:
        possiblecontaminants={}
        for tablepath in othermoleculepaths:
            othermolecule=tablepath.replace(f'{linemodelpath}','')
            othermolecule=othermolecule.replace('.fits','')
            
            othermoltable=QTable.read(tablepath)
            fwhm_Hz=velocitytofreq(fwhm_at_pix,reffreq)
            diff=np.abs(othermoltable['ReferenceFrequency']-reffreq)
            findcontam=np.where(diff<=fwhm_Hz)[0]
            if len(findcontam) > 0:
                candidate_contaminant_lines=othermoltable[findcontam]
                numcontaminants=len(candidate_contaminant_lines)
                if numcontaminants > 1:
                    contaminantflux=np.sum(candidate_contaminant_lines['ModelBrightness'])
                elif numcontaminants == 1:
                    contaminantflux=candidate_contaminant_lines['ModelBrightness']
                if contaminantflux >= (flux*minimumcontaminantthreshold):
                    print(f'{targetmolecule}\'s {qn} @ {reffreq} likely contaminated by {numcontaminants} {othermolecule} lines')
                    possiblecontaminants.update({othermolecule:candidate_contaminant_lines})
        if len(possiblecontaminants) == 0:
            if firstline:
                safelines=QTable(targetline)
                firstline=False
            else:
                safelines.add_row(targetline)

C2H5OH's 13(1,13)(1)-12(0,12)(1) @ 217.4561097355343 GHz likely contaminated by 3 CH2CHCN lines
C2H5OH's 30(3,27)(2)-30(2,28)(2) @ 219.13286692031934 GHz likely contaminated by 1 HC3Nv7_1 lines
C2H5OH's 15(5,10)(2)-15(4,11)(2) @ 232.03257142542915 GHz likely contaminated by 1 13CH3CN lines
C2H5OH's 14(5,9)(2)-14(4,10)(2) @ 232.8850927302058 GHz likely contaminated by 1 CH3OH lines
C2H5OH's 24(5,20)(2)-24(4,21)(2) @ 233.16505671692227 GHz likely contaminated by 3 CH3CH2CN lines
C2H5OH's 15(5,11)(2)-15(4,12)(2) @ 233.1720148192362 GHz likely contaminated by 2 CH3OCHO lines
C2H5OH's 14(5,10)(2)-14(4,11)(2) @ 233.55799583377697 GHz likely contaminated by 2 CH3COCH3 lines


In [4]:
doublets=[]
for targetline in safelines:
    flux=targetline['ModelBrightness']
    reffreq=targetline['ReferenceFrequency']
    qn=targetline['QNs']
    fwhm_Hz=velocitytofreq(fwhm_at_pix,reffreq)
    diff=np.abs(targetline['ReferenceFrequency']-reffreq)
    findcontam=np.where(diff<=fwhm_Hz)[0]
    if len(findcontam) > 1:
        print(f'{qn} has {len(findcontam)} companion lines')
        doublets.append(safelines[findcontam])
if len(doublets) == 0:
    print(f'{targetmolecule} has no doublets')

C2H5OH has no doublets


Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.


In [13]:
safelinepathbase=linemodelpath+'safelines/'
safelinepath=safelinepathbase+f'{targetmolecule}.fits'
if not os.path.exists(safelinepath):
    os.mkdir(safelinepathbase)
safelines.write(safelinepath)

In [47]:
othermoltable

QNs,RestFrequency,ReferenceFrequency,Eupper,Degeneracy,Aij,ModelBrightness,ModelNupper
,GHz,GHz,K,,Hz,K,1 / cm2
bytes17,float64,float64,float64,int64,float64,float64,float64
.222.0-.1.0,216.9367119,216.8962607473706,133.9354946499209,47,0.0008281647198900014,9.383164727233385,6545472842166.473
23.2.0-22.2.0,216.9367136,216.8962624470536,133.93549473150804,45,0.0008281367794532486,8.991088431623592,6266942080860.867
23.222.0-22.221.0,216.9367182,216.89626704619587,133.93549495227322,49,0.0008296950827092735,9.791976497494547,6824003593076.821
.222.1-.1.1,217.49757380000003,217.45701806613067,462.6187608395254,47,0.0008345190509189161,2.5312697306003953,1736119551982.6829
23.2.1-22.2.1,217.4975756,217.45701986579502,462.6189048035995,45,0.0008344912966901088,2.4240152402384516,1662241158007.0735
23.222.1-22.221.1,217.4975803,217.45702456491864,462.6189050291639,49,0.0008362540900265289,2.643848396283523,1809995925959.2317
.717.0-.716.0,218.3985275,218.35780376959696,231.53328525314117,47,0.0007729823468457445,5.870401138018595,4413658825833.371
.716.0-.715.0,218.39852770000002,218.35780396955968,231.53328526273964,47,0.0007729823475659071,5.870401132589456,4413658825662.317


In [48]:
reffreq

<Quantity 217.45610974 GHz>

MaskedNDArray(True)